# Extract Atmospec Centroid information

 work with Weakly_2022_39
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/11/02
- update : 2022/11/02


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/drp_pipe 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup
summit_utils          LOCAL:/home/d/dagoret/repos/repos_w_2022_39/summit_utils 	setup


In [2]:
! eups list -s lsst_distrib

   g0b29ad24fb+a10408d0bf 	current w_2022_39 setup


In [3]:
import lsst.daf.butler as dafButler
import lsst.summit.utils.butlerUtils as butlerUtils
import lsst.daf.base as dafBase

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

# Pipeline tasks from DP0.2
#from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
#from lsst.meas.algorithms.detection import SourceDetectionTask
#from lsst.meas.deblender import SourceDeblendTask
#from lsst.meas.base import SingleFrameMeasurementTask

# fromm StackClub on Britter-Fatter
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
import lsst.meas.extensions.shapeHSM
from lsst.pipe.tasks.calibrate import CalibrateTask, CalibrateConfig

In [6]:
#Set the matplotlib plot color table.
plt.style.use('tableau-colorblind10')

In [7]:
# Let us also set `lsst.afw.display` to use the `matplotlib` backend:
afwDisplay.setDefaultBackend('matplotlib')
plt.rcParams['figure.figsize'] = (6.0, 6.0)

In [8]:
repo =  "/sdf/group/rubin/repo/main"
#repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

## Give the collection

In [9]:
my_collection = "u/dagoret/spectro/noflat/empty~holo4/20211103"

In [10]:
#butler = butlerUtils.makeDefaultLatissButler(extraCollections=[my_collection])

## Config

In [11]:
# path index for each month
DATE="20211103"
filterdispersername = "empty~holo4_003"
#filterdispersername = "BG40~holo4_003"
#filterdispersername = "FELH0600~holo4_003"

## Select the atmospecCentroid

In [12]:
datasetRefs = registry.queryDatasets(datasetType='atmospecCentroid', collections=my_collection, where= "instrument='LATISS'")

In [13]:
all_dataId = []
all_atmcentroid  = []
all_exposures = []



for i, ref in enumerate(datasetRefs):
   
    print(f"========({i})================datasetType = atmospecCentroid ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_exposure = ref.dataId["visit"]
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0}
    print(the_dataId)
    #spec       = butler.get('spectraction',the_dataId)
    atmcentroid = butler.get('atmospecCentroid', visit=the_exposure, detector=0, collections=my_collection, instrument='LATISS')
    all_dataId.append(the_dataId) 
    all_exposures.append(the_exposure)
    all_atmcentroid.append(atmcentroid )
    #if i >= 0:
    #    break

========(0)================datasetType = atmospecCentroid ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2021110300459}
visit................: 2021110300459
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/spectro/noflat/empty~holo4/20211103/20221031T154824Z
{'day_obs': 20211103, 'seq_num': 459, 'detector': 0}
========(1)================datasetType = atmospecCentroid ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2021110300465}
visit................: 2021110300465
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/spectro/noflat/empty~holo4/20211103/20221031T155742Z
{'day_obs': 20211103, 'seq_num': 465, 'detector': 0}
========(2)=======

In [14]:
all_exposures

[2021110300459,
 2021110300465,
 2021110300471,
 2021110300477,
 2021110300483,
 2021110300489,
 2021110300495,
 2021110300501,
 2021110300507,
 2021110300513,
 2021110300519,
 2021110300525,
 2021110300531,
 2021110300537,
 2021110300543,
 2021110300556,
 2021110300562,
 2021110300568,
 2021110300574,
 2021110300580,
 2021110300586,
 2021110300592,
 2159388518340000,
 2021110300598,
 2159388518340000,
 2021110300604,
 2159388518340000,
 2021110300610,
 2021110300691,
 2021110300697,
 2021110300703,
 2021110300709,
 2021110300715,
 2021110300721,
 2021110300727,
 2021110300733,
 2021110300739]

In [16]:
all_atmcentroid

[{'astrometricMatch': False, 'centroid': [1700.0, 311.0]},
 {'astrometricMatch': False, 'centroid': [1751.0, 292.0]},
 {'astrometricMatch': False, 'centroid': [1752.0, 296.0]},
 {'astrometricMatch': False, 'centroid': [1753.0, 312.0]},
 {'astrometricMatch': False, 'centroid': [1705.0, 313.0]},
 {'astrometricMatch': False, 'centroid': [1709.0, 290.0]},
 {'astrometricMatch': False, 'centroid': [1757.0, 294.0]},
 {'astrometricMatch': False, 'centroid': [1757.0, 294.0]},
 {'astrometricMatch': False, 'centroid': [1761.0, 298.0]},
 {'astrometricMatch': False, 'centroid': [1761.0, 294.0]},
 {'astrometricMatch': False,
  'centroid': [1743.5735847479496, 1611.8931842877741]},
 {'astrometricMatch': False, 'centroid': [1770.0, 298.0]},
 {'astrometricMatch': False, 'centroid': [1775.0, 299.0]},
 {'astrometricMatch': False, 'centroid': [1778.0, 309.0]},
 {'astrometricMatch': False, 'centroid': [1781.0, 313.0]},
 {'astrometricMatch': False, 'centroid': [1705.0, 313.0]},
 {'astrometricMatch': False, 